In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

data = [[i for i in range(1,27)]]
columns = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
df = spark.createDataFrame(data, columns)
df.show()

df2 = spark.read.csv("dbfs:/FileStore/tables/dataset2.csv",header=True,inferSchema=True)
df2.show()

df.crossJoin(df2).show()


+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|  A|  B|  C|  D|  E|  F|  G|  H|  I|  J|  K|  L|  M|  N|  O|  P|  Q|  R|  S|  T|  U|  V|  W|  X|  Y|  Z|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23| 24| 25| 26|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| A1| B1| C1| D1| E1| F1| G1| H1| I1| J1| K1| L1| M1| N1| O1| P1| Q1| R1| S1| T1|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20|
|  2|  3|  4|  5|  6|  7|  8|  9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21|
|  3|  4|  5|  6|  7|  8|  9| 10| 11| 12| 13| 14| 15| 16| 1

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

df = spark.createDataFrame([
    ("January", "abc", 17), 
    ("February", "abc", 12), 
    ("March", "abc", 18), 
    ("April", "abc", 14), 
    ("June", "abc", 11), 
    ("August", "abc", 12), 
    ("September", "abc", 13), 
    ("November", "abc", 14), 
    ("December", "abc", 12)
], ["Month_Name", "CompanyName", "Profit"])

month_df = spark.createDataFrame([Row(Month="January"),
                                  Row(Month="February"),
                                  Row(Month="March"),
                                  Row(Month="April"),
                                  Row(Month="May"),
                                  Row(Month="June"),
                                  Row(Month="July"),
                                  Row(Month="August"),
                                  Row(Month="September"),
                                  Row(Month="October"),
                                  Row(Month="November"),
                                  Row(Month="December")])

df1 = month_df.join(df, month_df.Month==df.Month_Name,"left").orderBy("Month")

df_avg = df1.select(round(mean("Profit"),2).alias('Avg')).collect()
avg = df_avg[0]['Avg']
df1 = df1.withColumn("Profit",df1.Profit.cast(FloatType()))
df1 = df1.na.fill("abc",subset=["CompanyName"]).na.fill(avg,subset=["Profit"]).drop("Month_Name")
df1.show()

+---------+-----------+------+
|    Month|CompanyName|Profit|
+---------+-----------+------+
|    April|        abc|  14.0|
|   August|        abc|  12.0|
| December|        abc|  12.0|
| February|        abc|  12.0|
|  January|        abc|  17.0|
|     July|        abc| 13.67|
|     June|        abc|  11.0|
|    March|        abc|  18.0|
|      May|        abc| 13.67|
| November|        abc|  14.0|
|  October|        abc| 13.67|
|September|        abc|  13.0|
+---------+-----------+------+



In [0]:
df = spark.createDataFrame([("bench-1",1,"manufa",200,10,110,80,2021),("bench-1",1,"manufb",400,11,120,60,2021),("bench-2",1,"manufb",400,11,120,60,2021)],["Products","Soldto","CompanyName","Profit","x","y","z","Year"])

df.show()
df1=df.groupBy("Products").agg(collect_list(col("CompanyName")).alias("CompanyName"),sum("Profit"))
df1.show(truncate=False)

+--------+------+-----------+------+---+---+---+----+
|Products|Soldto|CompanyName|Profit|  x|  y|  z|Year|
+--------+------+-----------+------+---+---+---+----+
| bench-1|     1|     manufa|   200| 10|110| 80|2021|
| bench-1|     1|     manufb|   400| 11|120| 60|2021|
| bench-2|     1|     manufb|   400| 11|120| 60|2021|
+--------+------+-----------+------+---+---+---+----+

+--------+----------------+-----------+
|Products|CompanyName     |sum(Profit)|
+--------+----------------+-----------+
|bench-1 |[manufa, manufb]|600        |
|bench-2 |[manufb]        |400        |
+--------+----------------+-----------+

